In [50]:
#https://drive.google.com/file/d/1jbG9N1bMskmzRoIB0H_xaD2QOJclyeDC/view?usp=sharing
!gdown 1jbG9N1bMskmzRoIB0H_xaD2QOJclyeDC
#https://drive.google.com/file/d/1dzpej0QLLFvWlKKi7KwrzCOlQ0DJ2zB7/view?usp=sharing
!gdown 1dzpej0QLLFvWlKKi7KwrzCOlQ0DJ2zB7
#https://drive.google.com/file/d/1Zs1nLb0nkPWLgUSEhi1hsleulXaCtDCU/view?usp=sharing
!gdown 1Zs1nLb0nkPWLgUSEhi1hsleulXaCtDCU
#https://drive.google.com/file/d/1PQ_6Rtv_VdE_I04Nc-AQUdjVIAd2WBIk/view?usp=sharing
!gdown 1PQ_6Rtv_VdE_I04Nc-AQUdjVIAd2WBIk

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Downloading...
From: https://drive.google.com/uc?id=1jbG9N1bMskmzRoIB0H_xaD2QOJclyeDC
To: /content/danmu.csv
100% 124M/124M [00:01<00:00, 91.5MB/s]
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Downloading...
From: https://drive.google.com/uc?id=1dzpej0QLLFvWlKKi7KwrzCOlQ0DJ2zB7
To: /content/utils.py
100% 4.97k/4.97k [00:00<00:00, 4.02MB/s]
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Downloading...
From: https://drive.google.com/uc?id=1Zs1nLb0nkPWLgUSEhi1hsleulXaCtDCU
To: /content/category_info.txt
100% 5.56k/5.

In [51]:
!pip install bilibili_api

In [52]:
import re
import time
import jieba
import pickle
import asyncio
import datetime
import requests
import numpy as np
import pandas as pd 
import nest_asyncio
from utils import *
import jieba.analyse
from tqdm import tqdm
from os import listdir
from collections import Counter

jieba.initialize() 
nest_asyncio.apply()
# jieba.analyse.set_stop_words('stopwords.txt')

In [53]:
def child_to_parent(idx):
    if idx in cat_dic:
        name, __, parent = cat_dic[idx]
        return (name,parent)
    else:
        return None

In [54]:
cat_dic = init_category_dic()
par_ch_lst = []
cat_names = []
for i in list(cat_dic.keys()):
    name, tid = child_to_parent(i)
    if tid not in par_ch_lst: 
        par_ch_lst.append(tid)
        cat_names.append(name)
par_ch_lst = np.array(par_ch_lst)
cat_names = np.array(cat_names)

In [55]:
print(cat_dic[1])
print(par_ch_lst)
print(cat_names)
print(len(par_ch_lst))

('动画', 'douga', 1)
[  1  13 167   3 129   4  36 188 234 223 160 211 217 119 155 202   5 181
 177  23  11]
['动画' '番剧' '国创' '音乐' '舞蹈' '游戏' '知识' '科技' '运动' '汽车' '生活' '美食' '动物圈' '鬼畜'
 '时尚' '资讯' '娱乐' '影视' '纪录片' '电影' '电视剧']
21


In [56]:
def pre_p(name, option = 0):
    df = pd.read_csv('{}.csv'.format(name))

    f = open('stopwords.txt', encoding="utf8")
    stop_word = f.read().split('\n')
    f.close()
    
    size = 126 if option == 0 else len(par_ch_lst)
    cat_dic = init_category_dic()
    tid_lst = np.array(list(cat_dic.keys())) if option == 0 else par_ch_lst
    
    text_lst = df['danmu' if 'danmu' in name else 'comment'].tolist()
    fq_lst = df['Frequency'].tolist()
    ct_lst = df['Category ID'].tolist()
    
    token_dict = {}
    for idx in tqdm(range(len(text_lst))):
        tid = ct_lst[idx]
        if option != 0:
            tid = child_to_parent(tid)
            if tid == None:
                continue
            else:
              name, tid = tid
        if not pd.isna(text_lst[idx]) and tid in tid_lst: 
            lst_idx = np.where(tid_lst==tid)[0][0]
            seg_list = list(jieba.cut_for_search(text_lst[idx]))
            if len(text_lst[idx]) <= 6:
                seg_list.append(text_lst[idx])
            for token in seg_list:
                # token not in stop_word and 
                if len(token) > 2 and len(set([_ for _ in token])) > 1:
                    if token in token_dict:
                        tmp = np.eye(size)[lst_idx] * fq_lst[idx]
                        token_dict[token] = token_dict[token] + tmp
                    else:
                        token_dict[token] = np.eye(size)[lst_idx] * fq_lst[idx]
                        
    token_lst = np.array(list(token_dict.keys()))
    cnt_lst = np.array([sum(token_dict[i]) for i in token_lst])
    dist_lst = np.array(list(token_dict.values()))/cnt_lst[:,None]
    general_lst = np.array([len(np.where(i != 0)[0]) for i in dist_lst])
    
    df2 = pd.DataFrame(np.array([token_lst, cnt_lst, general_lst, ]).T, \
                   columns = ['Word Token', 'Total Apperance', 'Category Apperance'])
    return df2, dist_lst

In [57]:
danmu_df, danmu_dist = pre_p('danmu',1)

100%|██████████| 935917/935917 [02:41<00:00, 5787.15it/s]


In [58]:
print(danmu_df.head())
print(danmu_dist[3])
print(danmu_df.iloc[3])
print(len(danmu_dist[0]))

  Word Token Total Apperance Category Apperance
0      1人在观看             1.0                  1
1        进水管             1.0                  1
2       无能狂怒             9.0                  7
3        第一次          1050.0                 17
4       早啊各位             3.0                  3
[0.04       0.         0.01619048 0.08285714 0.00761905 0.15047619
 0.10285714 0.04       0.02857143 0.03238095 0.09428571 0.19714286
 0.01238095 0.00285714 0.01714286 0.         0.08857143 0.07238095
 0.01428571 0.         0.        ]
Word Token               第一次
Total Apperance       1050.0
Category Apperance        17
Name: 3, dtype: object
21


### Choose 5 categories out of 21, and add a 'general' category

In [59]:
import numpy as np
labels = [np.argmax(i) for i in danmu_dist]
chosen_cats = ['动画', '游戏', '科技', '生活', '美食']
chosen_cat_inds = [list(cat_names).index(i) for i in chosen_cats]
text_lst = danmu_df['Word Token'].to_list()
cate_apperance_lst = danmu_df['Category Apperance'].to_list()
print(chosen_cats)
print(chosen_cat_inds)

label_6_lst = []
text_6_lst = []
for ind, l in tqdm(enumerate(labels)):
    text = text_lst[ind]
    if text in ['nan', "None", 'Nan']:
        continue
    
    if int(cate_apperance_lst[ind]) > 3:
        
        text_6_lst.append(text)
        label_6_lst.append(5) #6 for general category. 
        
    elif l in chosen_cat_inds:
        
        text_6_lst.append(text)
        
        label = chosen_cat_inds.index(l)
        label_6_lst.append(label)
    
print(len(text_6_lst))
print(len(label_6_lst))

['动画', '游戏', '科技', '生活', '美食']
[0, 5, 7, 10, 11]


337392it [00:01, 187958.04it/s]

149981
149981


In [60]:
label_distribution = {}
for i in label_6_lst:
    if i in label_distribution:
        label_distribution[i] += 1
    else:
        label_distribution[i] = 1

print(label_distribution)

print(label_6_lst[:10])
text_6_lst[:10]

{0: 16438, 4: 38976, 5: 9903, 1: 41752, 2: 13504, 3: 29408}
[0, 4, 5, 5, 4, 4, 4, 5, 5, 4]


['1人在观看', '进水管', '无能狂怒', '第一次', '滑铁卢', '滑铁卢了', '尊严之战', '小老弟', '脑瓜子', '没有撒盐']

In [61]:
from sklearn.model_selection import train_test_split
import pandas as pd
df_6 = pd.DataFrame({
    'text':text_6_lst,
    'label':label_6_lst
})

train_df, test_df = train_test_split(df_6, test_size = 0.1)
train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [62]:
print(train_df.shape, test_df.shape, val_df.shape)

(107985, 2) (14999, 2) (26997, 2)


In [63]:
ls = [train_df, val_df, test_df]
ls2 = ['train', 'val', 'test']

for ind, df in enumerate(ls):
  texts = df['text'].to_list()
  labels = df['label'].to_list()
  with open(f'UER_ROBert_input_{ls2[ind]}.tsv', 'w+') as f:
      f.write('label')
      f.write('\t')
      f.write('text_a')
      f.write('\n')
      for ind in range(df.shape[0]):
          f.write(str(labels[ind]))
          f.write('\t')
          f.write(texts[ind])
          f.write('\n')
      f.close()

In [64]:
for i in ['danmu']:
    
    with open('{}_text_6_cates.pkl'.format(i), 'wb') as f:
        pickle.dump( text_6_lst , f)
        
        
    with open('{}_label_6_cates.pkl'.format(i), 'wb') as f:
        pickle.dump( label_6_lst, f)

In [65]:
# see_dist = np.array([sum([0 if (b[j] == 0)[i] else 1 for i in range(126)]) for j in range(len(b))])

In [66]:
danmu_df.sort_values(by=['Category Apperance'])[-15:]

,Word Token,Total Apperance,Category Apperance
10316,清晰度,39.0,9
43750,没多久,25.0,9
125880,太哈人,18.0,9
162072,很好看,24.0,9
23808,阿弥陀,38.0,9
23809,阿弥陀佛,65.0,9
354,太甜了,72.0,9
355,十块钱,55.0,9
26498,擦大哥,56.0,9
1606,chao,21.0,9


In [67]:
danmu_df.sort_values(by=['Total Apperance'])[-15:]

,Word Token,Total Apperance,Category Apperance
15573,我哭了,98.0,10
15903,看哭了,98.0,12
334191,已三连求资料,99.0,1
63458,好好看啊,99.0,11
51943,王侯将相,99.0,3
12380,三分钟,99.0,15
18190,蜘蛛侠,99.0,12
29797,cao,99.0,14
27451,律师函,99.0,11
3539,憋不住,99.0,16


In [68]:
comment_df.sort_values(by=['Category Apperance'])[-15:]

NameError: ignored

In [ ]:
comment_df.sort_values(by=['Total Apperance'])[-15:]